In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd gdrive/My\ Drive/KKB-kaggle/bengaliai-cv19/notebooks

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/My Drive/KKB-kaggle/bengaliai-cv19/notebooks


In [2]:
!pip install efficientnet_pytorch

In [0]:
# KaggleでEfficientNetを動かす場合

# %cd ../input/efficientnetdir/EfficientNet-PyTorch
# from efficientnet_pytorch import EfficientNet
# %cd ../..


In [0]:
## 諸々の import

import time
start = time.time()

import numpy as np
import pandas as pd
import cv2
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
import torchvision.models as models
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import PIL
import gc


from preprocess import *


import model.CNN as CNN
import model.efficientnet as efficientnet
import model.resnet18 as resnet18
import model.resnet34 as resnet34
import model.resnet50 as resnet50
import model.resnet101 as resnet101
import model.resnet152 as resnet152

In [0]:
## Parameters

# resize後のサイズ
HEIGHT = 64
WIDTH = 64

# 画像を3次元にするかどうか（EfficientNetなどを使うときはTrue）
# enable_3d = True
# ResNetとEfficientNetしか使わないので、常にTrueを想定する

# True なら submission.csv を生成する
create_submission = False

#Kaggleで提出するときはTrueにする
kaggle_flag = False

In [0]:
if kaggle_flag:
    dataset_dir = '/kaggle/input/bengaliai-cv19'
    model_dir = '/kaggle/input/trained_models'
else:
    dataset_dir = '../dataset'
    model_dir = '../trained_models'

# train_df = pd.read_csv(dataset_dir + '/train.csv')
test_df = pd.read_csv(dataset_dir + '/test.csv')
class_map_df = pd.read_csv(dataset_dir + '/class_map.csv')
sample_sub_df = pd.read_csv(dataset_dir + '/sample_submission.csv')


In [6]:
## アンサンブルの定義

model1 = resnet18.model()
model2 = efficientnet.model()


ensemble = [
    {
        'model': model1,
        'file': 'resnet18.pth',
        'w_grapheme': 0.30,
        'w_vowel': 0.30,
        'w_consonant': 0.30
    },
    {
        'model': model2,
        'file': 'efficientnet_28epoch.pth',
        'w_grapheme': 0.70,
        'w_vowel': 0.70,
        'w_consonant': 0.70
    }
]

# for M in ensemble としていないのは、ensembleに"代入"をしたいからです
# 参考：　https://ja.stackoverflow.com/q/31916
for i in range(len(ensemble)):
    device = torch.device('cpu')
    model = ensemble[i]['model']
    model.load_state_dict(torch.load(model_dir + '/' + ensemble[i]['file'], map_location=device))
    ensemble[i]['model'] = model
    print("model loaded from {}".format(model_dir + '/' + ensemble[i]['file']))


Loaded pretrained weights for efficientnet-b0
model loaded from ../trained_models/resnet18.pth
model loaded from ../trained_models/efficientnet_28epoch.pth


In [0]:
# テスト用のDataset, DataLoaderを作りたい

class TestDataset(torch.utils.data.Dataset):

    def __init__(self, X, transform=None, H=64, W=64):
        self.transform = transform
        self.X = X
        self.H = H 
        self.W = W 

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):

        out_data = cv2.resize(self.X[idx].reshape(self.H, self.W), (224, 224), interpolation=cv2.INTER_AREA)
        out_data = out_data.reshape(224, 224, 1).astype(np.uint8)
        out_data = cv2.cvtColor(out_data, cv2.COLOR_GRAY2RGB)
        out_data = np.transpose(out_data, (2,0,1)) / 255.0
        out_data = out_data.reshape(3,224,224) 
        
        out_data = torch.tensor(out_data, dtype=torch.float)

        if self.transform:
            out_data = self.transform(out_data)

        return out_data

In [8]:
## 提出ファイルの作成

target=[]
row_id=[] # row_id place holder

for parq_i in range(4):
    df_test_img = pd.read_parquet(dataset_dir + f'/test_image_data_{parq_i}.parquet')
    # df_test_img = pd.read_parquet(dataset_dir + f'/train_image_data_{parq_i}.parquet') # Error Check!
    df_test_img.set_index('image_id', inplace=True)

    X_test_resized = resize(df_test_img, out_height=HEIGHT, out_width=WIDTH).astype(np.uint8)
    
    index = 0
    id_names = df_test_img.index.values

    del df_test_img
    gc.collect()

    test_dataset = TestDataset(X_test_resized, H=HEIGHT, W=WIDTH)
    test_loader = DataLoader(test_dataset, batch_size=32, num_workers=0)

    del X_test_resized
    gc.collect()

    for data in test_loader:
        data = Variable(data)
        data = try_gpu(data)

        bs = len(data)
        root_ens, vowel_ens, consonant_ens = np.zeros((bs, 168)), np.zeros((bs, 11)), np.zeros((bs, 7))
        # ens = ensembleの意味を込めて

        for M in ensemble:
            model = M['model']
            model = try_gpu(model)

            root_o, vowel_o, consonant_o = model(data)
            root_ens += M['w_grapheme'] * root_o.data.cpu().numpy()
            vowel_ens += M['w_vowel'] * vowel_o.data.cpu().numpy()
            consonant_ens += M['w_consonant'] * consonant_o.data.cpu().numpy()

        root_pred, vowel_pred, consonant_pred = np.argmax(root_ens, axis=1), np.argmax(vowel_ens, axis=1), np.argmax(consonant_ens, axis=1)

        for i in range(bs):
            row_id.append(id_names[index]+'_consonant_diacritic')
            target.append(consonant_pred[i])
            row_id.append(id_names[index]+'_grapheme_root')
            target.append(root_pred[i])
            row_id.append(id_names[index]+'_vowel_diacritic')
            target.append(vowel_pred[i])
            index += 1
    
    del test_dataset
    del test_loader
    gc.collect()


df_sample = pd.DataFrame(
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)

if create_submission:
    df_sample.to_csv('submission.csv',index=False)

df_sample.head(36)


Resizing raw image... / 前処理実行中…
Resizing raw image... / 前処理実行中…
Resizing raw image... / 前処理実行中…
Resizing raw image... / 前処理実行中…
実行時間： 1415.2709159851074秒


In [9]:
elapsed_time = time.time() - start
print(f'実行時間： {elapsed_time}秒')

,row_id,target
0,Train_0_consonant_diacritic,5
1,Train_0_grapheme_root,15
2,Train_0_vowel_diacritic,9
3,Train_1_consonant_diacritic,0
4,Train_1_grapheme_root,159
